In [4]:
import requests
from bs4 import BeautifulSoup

In [14]:
def scrape_news(url, paragraph_tag = 'p', title_tag = 'h1'):
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        title = soup.find(title_tag).text
        
        content_list = [p.text for p in soup.find_all(paragraph_tag)]
        content = ' '.join([p.text for p in soup.find_all(paragraph_tag)])
        

        return {
            'title': title,
            'content': content
        }
    else:
        print(f"Failed to retrieve content from {url}")
        return None

In [19]:
def scrape_the_sun(url):
    return scrape_news(url, 'p', 'h1')

# def scrape_the_guardian(url):
#     return scrape_news(url, 'p', 'h1')

# def scrape_buzfeed(url):
#     return scrape_news(url, 'p', 'h1')


In [20]:
scrape_the_guardian('https://www.theguardian.com/uk-news/2023/oct/27/minister-denies-cultural-issue-among-tory-mps-after-crispin-blunts-arrest')['title']

'Minister denies ‘cultural issue’ among Tory MPs after Crispin Blunt’s arrest'

In [23]:
scrape_buzfeed('https://www.buzzfeednews.com/article/stephaniesoteriou/jimmy-kimmel-backlash-olivia-rodrigo-not-perfect?origin=hpdm?origin=spl')['title']

'Jimmy Kimmel Is Being Called Out After He “Unlocked” A “New Insecurity” For Olivia Rodrigo By Pointing Out A Minor Flaw And Saying She’s “Not Perfect After All” During A Live TV Interview'